In [121]:
pip install -q -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [122]:
import logging
import json
from IPython.display import Pretty, HTML
import urllib3

from pyfhirclient.FhirClient import FhirClient

logging.basicConfig()
logging.getLogger().setLevel(logging.ERROR)

# Disable the unverified HTTPS request warnings
urllib3.disable_warnings()

In [123]:
def setup_fhir_client() -> FhirClient:
    fhir_client = FhirClient(
        base_url='https://concept01.ehealthexchange.org:52780/fhirproxy/r4/',
        verify_ssl=False,
        extra_headers={
            'X-DESTINATION': 'CernerHelios',
            'X-POU': 'TREAT',
            'OAUTHSCOPES': 'system/Condition.read system/Encounter.read system/Immunization.read system/MedicationRequest.read system/Observation.read system/Patient.read system/Procedure.read system/MedicationAdministration.read system/DiagnosticReport.read system/RelatedPerson.read'
            # 'Host': 'concept01.ehealthexchange.org:52780'
        })

    fhir_client.auth_type = 'Basic'
    fhir_client.token = 'c3ZjX2VIeEZISVJTYW5kYm94OndpbGxmdWxTdHJvbmdTdGFuZHVyZDc'

    return fhir_client

In [124]:
client = setup_fhir_client()

In [135]:
search_results = client.search('Patient', {
    'address-postalcode': '64108'
})
# Pretty(json.dumps(search_results, indent=4))
patient_ids = []
for entry in search_results['entry']:
    patient_ids.append(entry['resource']['id'])
    
observations = []
for patient_id in patient_ids:
    observations.append(client.search('Observation', query_params={
        'patient': patient_id
    }))
Pretty(json.dumps(observations, indent=4))

[
    {
        "resourceType": "Bundle",
        "id": "ea031fe8-e815-4f11-ac8b-2deb35f15ae6",
        "type": "searchset",
        "link": [
            {
                "relation": "self",
                "url": "https://fhir-ehr.cerner.com/r4/e8a84236-c258-4952-98b7-a6ff8a9c587a/Observation?patient=12559994"
            }
        ],
        "entry": [
            {
                "fullUrl": "https://concept01.ehealthexchange.org:52780/fhirproxy/r4/Observation/SH-93-72797265",
                "resource": {
                    "resourceType": "Observation",
                    "id": "SH-93-72797265",
                    "meta": {
                        "versionId": "72797265",
                        "lastUpdated": "2018-01-24T21:54:52Z"
                    },
                    "text": {
                        "status": "generated",
                        "div": "<div xmlns=\"http://www.w3.org/1999/xhtml\"><p><b>Observation</b></p><p><b>Patient Id</b>: 12559994</p><p><b>Status

There should be exactly one `Patient` matching the search.

In [126]:
patient = search_results['entry'][0]
Pretty(json.dumps(patient, indent=4))

{
    "fullUrl": "https://concept01.ehealthexchange.org:52780/fhirproxy/r4/Patient/12559994",
    "resource": {
        "resourceType": "Patient",
        "id": "12559994",
        "meta": {
            "versionId": "3",
            "lastUpdated": "2018-02-02T15:11:47.000Z"
        },
        "text": {
            "status": "generated",
            "div": "<div xmlns=\"http://www.w3.org/1999/xhtml\"><p><b>Patient</b></p><p><b>Name</b>: NURSING, RESTRAINT</p><p><b>Status</b>: Active</p><p><b>DOB</b>: Dec 15, 1991</p><p><b>Administrative Gender</b>: Female</p><p><b>Marital Status</b>: Single</p></div>"
        },
        "extension": [
            {
                "extension": [
                    {
                        "valueCoding": {
                            "system": "urn:oid:2.16.840.1.113883.6.238",
                            "code": "2076-8",
                            "display": "Native Hawaiian or Other Pacific Islander",
                            "userSelected": fal

In [127]:
HTML(patient['resource']['text']['div'])

`MedicationAdministration` should have the medications administered to this Patient.

In [128]:
medication_administration_results = client.search('MedicationAdministration', query_params={
    'patient': patient['resource']['id']
})
Pretty(json.dumps(medication_administration_results, indent=4))

{
    "resourceType": "Bundle",
    "id": "e61c3377-e5a0-4069-a199-a004158dbba1",
    "type": "searchset",
    "link": [
        {
            "relation": "self",
            "url": "https://fhir-ehr.cerner.com/r4/e8a84236-c258-4952-98b7-a6ff8a9c587a/MedicationAdministration?patient=12559994"
        }
    ]
}

`Observation` should have lab test results, and pregnancy status.

In [133]:
observation_search_results = client.search('Observation', query_params={
    'patient': patient['resource']['id']
})

Pretty(json.dumps(observation_search_results, indent=4))        

{
    "resourceType": "Bundle",
    "id": "68272a77-2d6b-4c22-a67e-864597aa09d9",
    "type": "searchset",
    "link": [
        {
            "relation": "self",
            "url": "https://fhir-ehr.cerner.com/r4/e8a84236-c258-4952-98b7-a6ff8a9c587a/Observation?patient=12559994"
        }
    ],
    "entry": [
        {
            "fullUrl": "https://concept01.ehealthexchange.org:52780/fhirproxy/r4/Observation/SH-93-72797265",
            "resource": {
                "resourceType": "Observation",
                "id": "SH-93-72797265",
                "meta": {
                    "versionId": "72797265",
                    "lastUpdated": "2018-01-24T21:54:52Z"
                },
                "text": {
                    "status": "generated",
                    "div": "<div xmlns=\"http://www.w3.org/1999/xhtml\"><p><b>Observation</b></p><p><b>Patient Id</b>: 12559994</p><p><b>Status</b>: Final</p><p><b>Categories</b>: Social History</p><p><b>Code</b>: Tobacco</p><p><b>Issued

`Encounter` should have hospitalization status.

In [ ]:
encounter_search_results = client.search('Encounter', query_params={
    'patient': patient['resource']['id']
})
Pretty(json.dumps(encounter_search_results, indent=4))

In [ ]:
client.close()